In [ ]:
# Import Libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU, Reshape, Conv2D, Conv2DTranspose, BatchNormalization,Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import os
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.models import load_model

In [ ]:
# Loading the 'test' and 'validation' splits of the Dmlab dataset from TensorFlow datasets.
dmlab_test_data = tfds.load('dmlab', split='test')
dmlab_validation_data = tfds.load('dmlab', split='validation')


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/65550 [00:00<?, ? examples/s]

...

Generating validation examples...:   0%|          | 0/22628 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/dmlab/2.0.1.incompleteLYDOJX/dmlab-validation.tfrecord*...:   0%|         …

Generating test examples...:   0%|          | 0/22735 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/dmlab/2.0.1.incompleteLYDOJX/dmlab-test.tfrecord*...:   0%|          | 0/2…

Dataset dmlab downloaded and prepared to /root/tensorflow_datasets/dmlab/2.0.1. Subsequent calls will reuse this data.


In [ ]:
# Setting hyperparameters:
BATCH_SIZE = 8      # Number of training samples processed before the model is updated.
EPOCHS = 2000       # Number of complete passes through the dataset.
NOISE_DIM = 200     # Dimensionality of the random noise input for the generator.
SAVE_INTERVAL = 50  # Frequency (in terms of epochs) at which the model and generated images are saved.
TRAINING_RATIO = 5  # Number of discriminator updates per generator update.

In [ ]:
# Define the Wasserstein loss function.
def wasserstein_loss(y_true, y_pred):
    return tf.reduce_mean(y_true * y_pred)

In [ ]:
# Preprocessing function for the dataset:
def preprocess_dataset(dataset):
    def _preprocess_img(img):
        img = tf.image.resize(img, (256, 256))
        img = (img - 127.5) / 127.5
        return img

    return dataset.map(lambda x: _preprocess_img(x['image'])).batch(BATCH_SIZE)


In [ ]:
# Preprocess the test and validation datasets for DMLab:
dmlab_test_data = tfds.load('dmlab', split='test')
dmlab_validation_data = tfds.load('dmlab', split='validation')

dmlab_test_data_processed = preprocess_dataset(dmlab_test_data)
dmlab_validation_data_processed = preprocess_dataset(dmlab_validation_data)


In [ ]:
# Defines the generator model using Conv2DTranspose layers to upscale a noise vector into a 256x256 image.
def build_generator():
    model = Sequential()
    model.add(Dense(128 * 8 * 8, activation="relu", input_shape=(NOISE_DIM,)))
    model.add(Reshape((8, 8, 128)))
    model.add(BatchNormalization())

    # UpSample to 16x16
    model.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))

    # UpSample to 32x32
    model.add(Conv2DTranspose(32, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))

    # UpSample to 64x64
    model.add(Conv2DTranspose(16, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))

    # UpSample to 128x128
    model.add(Conv2DTranspose(8, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(3, kernel_size=4, strides=2, padding="same", activation="tanh"))
    return model

In [ ]:
# Defines the discriminator model using Conv2D layers to classify a 256x256 image as real or fake.
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(16, kernel_size=4, strides=2, padding="same", input_shape=(256, 256, 3)))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(32, kernel_size=4, strides=2, padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(64, kernel_size=4, strides=2, padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(128, kernel_size=4, strides=2, padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Flatten())
    model.add(Dense(1))
    return model

In [ ]:
# Initialise the generator and discriminator models.
generator = build_generator()
discriminator = build_discriminator()

In [ ]:
# Define optimisers - with specific learning rates and betas.
optimizer_gen = Adam(0.0001, beta_1=0.5, beta_2=0.9)
optimizer_disc = Adam(0.0004, beta_1=0.5, beta_2=0.9)

In [ ]:
# Compilation
generator.compile(optimizer=optimizer_gen, loss=wasserstein_loss)
discriminator.compile(optimizer=optimizer_disc, loss=wasserstein_loss)

# Combined model
z = tf.keras.layers.Input(shape=(NOISE_DIM,))
img = generator(z)
discriminator.trainable = False
validity = discriminator(img)
combined = Model(z, validity)
combined.compile(loss=wasserstein_loss, optimizer=optimizer_gen)


In [ ]:
def train_gan(dataset, epochs):
    valid = -np.ones((BATCH_SIZE, 1))
    fake = np.ones((BATCH_SIZE, 1))

    for epoch in range(epochs):
        for _ in range(TRAINING_RATIO):
            for imgs in dataset.take(1):
                noise = np.random.normal(0, 1, (BATCH_SIZE, NOISE_DIM))
                gen_imgs = generator.predict(noise)
                d_loss_real = discriminator.train_on_batch(imgs, valid)
                d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                for l in discriminator.layers:
                    weights = l.get_weights()
                    weights = [np.clip(w, -0.01, 0.01) for w in weights]
                    l.set_weights(weights)

        g_loss = combined.train_on_batch(noise, valid)
        print(f"{epoch}/{epochs} [D loss: {d_loss} | G loss: {g_loss}]")

        if epoch % SAVE_INTERVAL == 0:
            save_imgs(generator, discriminator, epoch)
            generator.save('generator_model.h5')
            discriminator.save('discriminator_model.h5')  # Save the discriminator model


In [ ]:
# Save image function
def save_imgs(generator, discriminator, epoch, save_path="GANimages", num_samples=25):
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    noise = np.random.normal(0, 1, (num_samples, NOISE_DIM))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5

    for i in range(num_samples):
        plt.imshow(gen_imgs[i])
        plt.axis('off')
        plt.savefig(f"{save_path}/image_at_epoch_{epoch}_sample_{i}.png")
        plt.close()


In [ ]:
train_gan(dmlab_validation_data_processed, EPOCHS)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 19ms/step
1169/2000 [D loss: -47.799245834350586 | G loss: 22.892465591430664]
1/1 [==============================] - 0s 20ms/step
1170/2000 [D loss: -38.25975799560547 | G loss: 16.937911987304688]
1/1 [==============================] - 0s 21ms/step
1171/2000 [D loss: -41.710819244384766 | G loss: 20.956907272338867]
1/1 [==============================] - 0s 19ms/step
1172/2000 [D loss: -40.216766357421875 | G loss: 21.472524642944336]
1/1 [==============================] - 0s 35ms/step
1173/2000 [D loss: -38.09988021850586 | G loss: 22.656824111938477]
1/1 [==============================] - 0s 29ms/step
1174/2000 [D loss: -43.60787010192871 | G loss: 25.350765228271484]
1/1 [==============================] - 0s 22ms/step
1175/2000 [D loss: -41.09834098815918 | G loss: 27.32927131652832]
1/1 [==============================] - 0s 19ms/step
1176/2000 [D loss: -39.668724060058594 | G loss: 31.638

In [ ]:
# Load the pre-trained discriminator model from the saved file.
with tf.keras.utils.custom_object_scope({'wasserstein_loss': wasserstein_loss}):
    discriminator = load_model('discriminator_model.h5')

In [ ]:
# Load the pre-trained generator model from the saved file.
with tf.keras.utils.custom_object_scope({'wasserstein_loss': wasserstein_loss}):
    generator = load_model('generator_model.h5')

In [ ]:
# Save images as a ZIP!
!zip -r GANimages.zip GANimages/

  adding: GANimages/ (stored 0%)
  adding: GANimages/image_at_epoch_400_sample_12.png (deflated 0%)
  adding: GANimages/image_at_epoch_1600_sample_13.png (deflated 0%)
  adding: GANimages/image_at_epoch_1300_sample_7.png (deflated 0%)
  adding: GANimages/image_at_epoch_1450_sample_20.png (deflated 0%)
  adding: GANimages/image_at_epoch_1350_sample_6.png (deflated 0%)
  adding: GANimages/image_at_epoch_850_sample_13.png (deflated 0%)
  adding: GANimages/image_at_epoch_300_sample_15.png (deflated 0%)
  adding: GANimages/image_at_epoch_200_sample_6.png (deflated 0%)
  adding: GANimages/image_at_epoch_1150_sample_6.png (deflated 0%)
  adding: GANimages/image_at_epoch_950_sample_11.png (deflated 0%)
  adding: GANimages/image_at_epoch_450_sample_1.png (deflated 0%)
  adding: GANimages/image_at_epoch_50_sample_22.png (deflated 1%)
  adding: GANimages/image_at_epoch_1800_sample_11.png (deflated 0%)
  adding: GANimages/image_at_epoch_1850_sample_3.png (deflated 0%)
  adding: GANimages/image_at_

In [ ]:
# Download the ZIP! folder
from google.colab import files
files.download("GANimages.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>